#Estructura de Proyecto
Se divide en tres jupyter notebooks:

Creación de base de datos

Analisis exploratorio de los  datos (EDA)

Modelando Redes Neuronales

#Creación de base de datos

In [ ]:
#Importando Librerias

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import json
from sqlite3 import dbapi2 as sq3
from pathlib import Path
from collections import OrderedDict

# importar tensorflow as tf
# from tensorflow import keras

from time import time
from IPython.display import clear_output

import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Dataset Doc: https://www.yelp.com/dataset/documentation/main
#Data Paths
REVIEW_PATH = 'review.json'      #Contiene datos completos del texto de la reseña, incluido el ID de usuario que escribió la reseña y el ID de empresa para el que se escribió la reseña.
CHECKIN_PATH = 'checkin.json'    #Checkins de business.
BUSINESS_PATH = 'business.json'  #Contiene datos comerciales, incluidos datos de ubicación, atributos y categorías.
TIP_PATH = 'tip.json'            #Consejos escritos por un usuario sobre un negocio. Los consejos son más breves que las reseñas y tienden a transmitir sugerencias rápidas.
USER_PATH = 'user.json'          #Datos del usuario, incluida la asignación de amigos del usuario y todos los metadatos asociados con el usuario.

# Funciones para cargar datos desde formato json

def load_rows(file_path, nrows=None, only_return_count=False, verbose=True):
    """
    Devuelve el marco de datos del archivo json
    """
    tic = time()
    with open(file_path) as json_file:
        count = 0
        objs = []
        line = json_file.readline()
        while (nrows is None or count<nrows) and line:
            count += 1
            if not only_return_count:
                obj = json.loads(line)
                objs.append(obj)
            line = json_file.readline()
        toc = time()
        if verbose:
            print(file_path.split('/')[-1], 'loaded. Count =', count, ', Time =', round(toc-tic,2), 'secs.')

        if only_return_count:
            return count

        return pd.DataFrame(objs)


#generador de datos para cargar datos en trozos
def load_rows_gen(file_path, nrows=1e6, verbose=True):
    """
Devuelve datos en trozos
    """
    with open(file_path) as json_file:
        line = json_file.readline()
        total = 0
        while line:
            count = 0
            objs = []
            tic = time()
            while count<nrows and line:
                count+=1
                obj = json.loads(line)
                objs.append(obj)
                line = json_file.readline()
                total += count
            toc = time()
            print('Loaded chunk of size:', count, ", Time =", round(toc-tic,2), 'secs.')
            yield pd.DataFrame(objs)

In [ ]:
user_df_head = load_rows(USER_PATH, 5)
user_df_head

user.json loaded. Count = 5 , Time = 0.21 secs.


,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,2007-01-25 16:47:26,7217,1259,5994,2007,"NSCy54eWehBJyZdG2iE84w, pe42u7DcCH2QmI81NX-8qA...",267,...,65,55,56,18,232,844,467,467,239,180
1,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,2009-01-25 04:35:42,43091,13066,27281,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2...","ueRPE0CX75ePGMqOFVj6IQ, 52oH4DrRvzzl8wh5UXyU0A...",3138,...,264,184,157,251,1847,7054,3131,3131,1521,1946
2,2WnXYQFK0hXEoTxPtV2zvg,Steph,665,2008-07-25 10:41:00,2086,1010,1003,"2009,2010,2011,2012,2013","LuO3Bn4f3rlhyHIaNfTlnA, j9B4XdHUhDfTKVecyWQgyA...",52,...,13,10,17,3,66,96,119,119,35,18
3,SZDeASXq7o05mMNLshsdIA,Gwen,224,2005-11-29 04:38:33,512,330,299,"2009,2010,2011","enx1vVPnfdNUdPho6PH_wg, 4wOcvMLtU6a9Lslggq74Vg...",28,...,4,1,6,2,12,16,26,26,10,9
4,hA5lMy-EnncsH4JoR-hFGQ,Karen,79,2007-01-05 19:40:59,29,15,7,,"PBK4q9KEEBHhFvSXCUirIw, 3FWPpM7KU1gXeOM_ZbYMbA...",1,...,1,0,0,0,1,1,0,0,0,0


In [ ]:
review_df_head = load_rows(REVIEW_PATH, 5)
review_df_head

review.json loaded. Count = 5 , Time = 0.02 secs.


,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5.0,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3.0,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4.0,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15


In [ ]:
checkin_df_head = load_rows(CHECKIN_PATH, 5)
checkin_df_head

checkin.json loaded. Count = 5 , Time = 0.04 secs.


,business_id,date
0,---kPU91CF4Lq2-WlRu9Lw,"2020-03-13 21:10:56, 2020-06-02 22:18:06, 2020..."
1,--0iUa4sNDFiZFrAdIWhZQ,"2010-09-13 21:43:09, 2011-05-04 23:08:15, 2011..."
2,--30_8IhuyMHbSOcNWd6DQ,"2013-06-14 23:29:17, 2014-08-13 23:20:22"
3,--7PUidqRWpRSpXebiyxTg,"2011-02-15 17:12:00, 2011-07-28 02:46:10, 2012..."
4,--7jw19RH9JKXgFohspgQw,"2014-04-21 20:42:11, 2014-04-28 21:04:46, 2014..."


In [ ]:
tip_df_head = load_rows(TIP_PATH, 5)
tip_df_head

tip.json loaded. Count = 5 , Time = 0.04 secs.


,user_id,business_id,text,date,compliment_count
0,AGNUgVwnZUey3gcPCJ76iw,3uLgwr0qeCNMjKenHJwPGQ,Avengers time with the ladies.,2012-05-18 02:17:21,0
1,NBN4MgHP9D3cw--SnauTkA,QoezRbYQncpRqyrLH6Iqjg,They have lots of good deserts and tasty cuban...,2013-02-05 18:35:10,0
2,-copOvldyKh1qr-vzkDEvw,MYoRNLb5chwjQe3c_k37Gg,It's open even when you think it isn't,2013-08-18 00:56:08,0
3,FjMQVZjSqY8syIO-53KFKw,hV-bABTK-glh5wj31ps_Jw,Very decent fried chicken,2017-06-27 23:05:38,0
4,ld0AperBXk1h6UbqmM80zw,_uN0OudeJ3Zl_tf6nxg5ww,Appetizers.. platter special for lunch,2012-10-06 19:43:09,0


In [ ]:
business_df_head = load_rows(BUSINESS_PATH, 5)
business_df_head.head()

business.json loaded. Count = 5 , Time = 0.15 secs.


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."


In [ ]:
# Mirando una descripción general de nuestros datos
business_df_head.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   business_id   5 non-null      object 
 1   name          5 non-null      object 
 2   address       5 non-null      object 
 3   city          5 non-null      object 
 4   state         5 non-null      object 
 5   postal_code   5 non-null      object 
 6   latitude      5 non-null      float64
 7   longitude     5 non-null      float64
 8   stars         5 non-null      float64
 9   review_count  5 non-null      int64  
 10  is_open       5 non-null      int64  
 11  attributes    5 non-null      object 
 12  categories    5 non-null      object 
 13  hours         4 non-null      object 
dtypes: float64(3), int64(2), object(9)
memory usage: 688.0+ bytes


In [ ]:
#Usaremos datos de Usuario (detalles sobre el usuario), Negocio (Detalles sobre el negocio) y Revisión (usuarios y sus reseñas de negocios)..

In [ ]:
# Funciones para trabajar con db2api desde sqlite3

PATHSTART = "."
def get_db(dbfile):
    #obtener conexión a db
    sqlite_db = sq3.connect(Path(PATHSTART)/ dbfile)
    return sqlite_db

def init_db(dbfile, schema):
    #crear db a/c al esquema
    db = get_db(dbfile)

    #ejecutar código sql
    c = db.cursor()
    c.executescript(schema)

    #hacer commit
    db.commit()
    return db

def make_query(sel):
    c = db.cursor().execute(sel)
    return c.fetchall()

from collections import OrderedDict
def make_frame(list_of_tuples, legend):
    framelist=[]
    for i, cname in enumerate(legend):
        framelist.append((cname,[e[i] for e in list_of_tuples]))
    return pd.DataFrame.from_dict(OrderedDict(framelist))

In [ ]:
#Esquema de tablas para tablas en nuestra base de datos SQLite

users_schema = """
DROP TABLE IF EXISTS "users";

CREATE TABLE "users" (
    "user_id" INTEGER PRIMARY KEY NOT NULL,
    "name" VARCHAR,
    "review_count" INTEGER,
    "yelping_since" TIMESTAMP,
    "useful" INTEGER,
    "funny" INTEGER,
    "cool" INTEGER,
    "elite" VARCHAR,
    "friends" VARCHAR,
    "fans" INTEGER,
    "average_stars" FLOAT,
    "compliment_hot" INTEGER,
    "compliment_more" INTEGER,
    "compliment_profile" INTEGER,
    "compliment_cute" INTEGER,
    "compliment_list" INTEGER,
    "compliment_note" INTEGER,
    "compliment_plain" INTEGER,
    "compliment_cool" INTEGER,
    "compliment_funny" INTEGER,
    "compliment_writer" INTEGER,
    "compliment_photos" INTEGER
);
"""
businesses_schema="""
DROP TABLE IF EXISTS "businesses";

CREATE TABLE "businesses" (
    "business_id" INTEGER PRIMARY KEY NOT NULL,
    "name" VARCHAR,
    "address" VARCHAR,
    "city" VARCHAR,
    "state" VARCHAR,
    "postal_code" VARCHAR,
    "latitude" FLOAT,
    "longitude" FLOAT,
    "stars" FLOAT,
    "review_count" INTEGER,
    "is_open" BOOLEAN,
    "categories" VARCHAR,
"""
reviews_schema = """
DROP TABLE IF EXISTS "reviews";

CREATE TABLE "reviews" (
    "review_id" VARCHAR PRIMARY KEY,
    "user_id" INTEGER,
    "business_id" INTEGER,
    "stars" FLOAT,
    "useful" INTEGER,
    "funny" INTEGER,
    "cool" INTEGER,
    "text"  VARCHAR,
    "date" TIMESTAMP,


    FOREIGN KEY (user_id) REFERENCES users(user_id),
    FOREIGN KEY (business_id) REFERENCES businesses(business_id)
);
"""
schema_close = ");"

In [ ]:
#Cargar datos sobre todas las empresas.
business_EEUU = load_rows(BUSINESS_PATH)
business_EEUU.head()

business.json loaded. Count = 150346 , Time = 5.36 secs.


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."


In [ ]:
business_df = business_EEUU[business_EEUU['state'].str.contains('FL')]

In [ ]:
business_df

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
7,qkRM_2X51Yqxk3btlwAQIg,Temple Beth-El,400 Pasadena Ave S,St. Petersburg,FL,33707,27.766590,-82.732983,3.5,5,1,None,"Synagogues, Religious Organizations","{'Monday': '9:0-17:0', 'Tuesday': '9:0-17:0', ..."
10,UJsufbvfyfONHeWdvAHKjA,Marshalls,21705 Village Lakes Sc Dr,Land O' Lakes,FL,34639,28.190459,-82.457380,3.5,6,1,"{'RestaurantsPriceRange2': '2', 'BikeParking':...","Department Stores, Shopping, Fashion","{'Monday': '9:30-21:30', 'Tuesday': '9:30-21:3..."
11,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,,Tampa Bay,FL,33602,27.955269,-82.456320,4.0,10,1,"{'Alcohol': ''none'', 'OutdoorSeating': 'None'...","Vietnamese, Food, Restaurants, Food Trucks","{'Monday': '11:0-14:0', 'Tuesday': '11:0-14:0'..."
13,jaxMSoInw8Poo3XeMJt8lQ,Adams Dental,15 N Missouri Ave,Clearwater,FL,33755,27.966235,-82.787412,5.0,10,1,{'ByAppointmentOnly': 'True'},"General Dentistry, Dentists, Health & Medical,...","{'Monday': '7:30-15:30', 'Tuesday': '7:30-15:3..."
14,0bPLkL0QhhPO5kt1_EXmNQ,Zio's Italian Market,2575 E Bay Dr,Largo,FL,33771,27.916116,-82.760461,4.5,100,0,"{'OutdoorSeating': 'False', 'RestaurantsGoodFo...","Food, Delis, Italian, Bakeries, Restaurants","{'Monday': '10:0-18:0', 'Tuesday': '10:0-20:0'..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150280,37G3SzO7RS1qSthACCG5SQ,Tampa Bay Club Sport,380 105th Terrace Ne,Saint Petersburg,FL,33716,27.867754,-82.632602,2.5,7,0,{'GoodForKids': 'True'},"Active Life, Sports Clubs",None
150289,Fck8i0fNQCa22ERz5Fa21w,Thoughtful Moving,5004 E Fowler Ave,Tampa,FL,33617,28.054934,-82.400832,2.0,27,1,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Packing Services, Home Services, Movers, Local...","{'Monday': '22:0-22:30', 'Tuesday': '8:0-19:0'..."
150292,esBGrrmuZzSiECyRBoKvvA,Colony Grill - St. Petersburg,670 Central Ave,St. Petersburg,FL,33701,27.770872,-82.643069,4.5,38,1,"{'RestaurantsPriceRange2': '2', 'RestaurantsAt...","Bars, Beer Bar, Nightlife, Wine Bars, Pizza, R...","{'Monday': '11:30-23:0', 'Tuesday': '11:30-23:..."
150317,Q7JYAMNzI1IpUd2edflmTA,21 Barber,10937 56th St N,Temple Terrace,FL,33617,28.047632,-82.393519,4.5,18,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Men's Hair Salons, Hair Salons, Barbers, Beaut...","{'Monday': '9:0-20:0', 'Tuesday': '9:0-20:0', ..."


In [ ]:
#Aquí preprocesamos los datos de nuestras empresas.
def preprocess_business_df(df):
    """
    Preprocesamos data de BUSINESS_PATH
    returna final DataFrame
    """
    #máscara para seleccionar solo negocios que sean saludables o sirvan alimentos
    mask = df.categories.str.contains('Health', na=False) | df.categories.str.contains('Food', na=False)
    df = df[mask]
    #Cambiando business_id a números
    global businessid_to_idx
    businessid_to_idx = {b_id : idx for idx, b_id in enumerate(df.business_id.unique())}
    df.business_id = df.business_id.map(lambda x: businessid_to_idx[x])

    #TDT
    df.is_open = df.is_open.astype(bool)

    # Explotanto atributos [MultiCategorizacion]
    attr = [col for col in df.attributes.explode().unique() if col is not None]
    lst_of_attr_dict = []
    for attr_dict in df.attributes:
        if not attr_dict:
            lst_of_attr_dict.append({})
            continue

        if 'BusinessParking' in attr_dict:
            if type(attr_dict['BusinessParking']) == str:
                attr_dict['BusinessParking'] = ('True' in attr_dict['BusinessParking'])

        lst_of_attr_dict.append(attr_dict)

    attr_df = pd.DataFrame(lst_of_attr_dict, columns=attr)
    for col in attr_df:
        #manejo de errores
        ##Estrategia -> la ausencia de un atributo significa que la empresa no lo tiene
        #ex. Si el estacionamiento es nulo, entonces el negocio no tiene estacionamiento.
        attr_df[col] = attr_df[col].fillna(False).astype(bool)

    df = pd.concat([df.reset_index().drop('index', axis=1), attr_df], axis=1)
    df.drop(['attributes'], axis=1, inplace=True)

    #Horas explosivas, es decir. obtener hora de apertura y cierre para varios días
    lst_of_time = []
    for time_dict in df.hours:
        if not time_dict:
            lst_of_time.append({})
            continue
        lst_of_time.append(time_dict)
    time_df = pd.DataFrame(lst_of_time)
    df = pd.concat([df, time_df], axis=1).drop('hours', axis=1)

    return df

business_df = preprocess_business_df(business_df)
business_df

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,AgesAllowed,Open24Hours,DietaryRestrictions,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,0,Vietnamese Food Truck,,Tampa Bay,FL,33602,27.955269,-82.456320,4.0,10,...,False,False,False,11:0-14:0,11:0-14:0,11:0-14:0,11:0-14:0,11:0-14:0,5:0-10:0,15:0-18:0
1,1,Adams Dental,15 N Missouri Ave,Clearwater,FL,33755,27.966235,-82.787412,5.0,10,...,False,False,False,7:30-15:30,7:30-15:30,7:30-15:30,7:30-15:30,NaN,NaN,NaN
2,2,Zio's Italian Market,2575 E Bay Dr,Largo,FL,33771,27.916116,-82.760461,4.5,100,...,False,False,False,10:0-18:0,10:0-20:0,10:0-20:0,10:0-20:0,10:0-20:0,10:0-20:0,NaN
3,3,Charlie's Market,2815 E Sligh Ave,Tampa,FL,33610,28.010360,-82.430042,3.0,9,...,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Publix Super Market,12101 Little Rd,Hudson,FL,34667,28.332601,-82.668107,3.5,7,...,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7965,7965,Johnny Grits Southern Fresh,5749 Main St,New Port Richey,FL,34652,28.250360,-82.718935,3.5,224,...,False,False,False,7:0-14:0,7:0-14:0,7:0-14:0,7:0-14:0,7:0-14:0,7:0-14:0,8:0-14:0
7966,7966,Subway,4882 Sun City Center Blvd,Sun City Center,FL,33573,27.714605,-82.335851,4.0,5,...,False,False,False,0:0-0:0,9:0-21:0,9:0-21:0,8:0-22:0,8:0-22:0,9:0-22:0,9:0-21:0
7967,7967,Speedway,28232 Wesley Chapel Blvd,Wesley Chapel,FL,33543,28.236004,-82.349088,2.5,7,...,False,False,False,0:0-0:0,0:0-0:0,0:0-0:0,0:0-0:0,0:0-0:0,0:0-0:0,0:0-0:0
7968,7968,Sweet L's Bakery,26 Hibiscus St,Tarpon Springs,FL,34689,28.147221,-82.756020,5.0,10,...,False,False,False,0:0-0:0,NaN,11:0-16:0,11:0-16:0,11:0-16:0,11:0-15:0,NaN


In [ ]:
# Mirando una descripción general de nuestros datos
business_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7970 entries, 0 to 7969
Data columns (total 58 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   business_id                 7970 non-null   int64  
 1   name                        7970 non-null   object 
 2   address                     7970 non-null   object 
 3   city                        7970 non-null   object 
 4   state                       7970 non-null   object 
 5   postal_code                 7970 non-null   object 
 6   latitude                    7970 non-null   float64
 7   longitude                   7970 non-null   float64
 8   stars                       7970 non-null   float64
 9   review_count                7970 non-null   int64  
 10  is_open                     7970 non-null   bool   
 11  categories                  7970 non-null   object 
 12  Alcohol                     7970 non-null   bool   
 13  OutdoorSeating              7970 

In [ ]:
# Completando... el esquema de la tabla de negocios
for bool_col in business_df.columns[12:51]:
    businesses_schema += '    \"' + bool_col + '\"' + ' BOOLEAN,\n'
for day in business_df.columns[51:]:
    businesses_schema += '    \"' + day + '\"' + ' VARCHAR,\n'

businesses_schema = businesses_schema[:-2] + schema_close

In [ ]:
#Creando db
db = init_db("yelp_database.db", users_schema+businesses_schema+reviews_schema)

#datos comerciales a sql
business_df.to_sql('businesses', db, if_exists='append', index=False)

#liberar memoria
del business_df

In [ ]:
#cargar datos sobre los usuarios
user_df = load_rows(USER_PATH)
user_df.head()

user.json loaded. Count = 1987897 , Time = 99.72 secs.


,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,2007-01-25 16:47:26,7217,1259,5994,2007,"NSCy54eWehBJyZdG2iE84w, pe42u7DcCH2QmI81NX-8qA...",267,...,65,55,56,18,232,844,467,467,239,180
1,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,2009-01-25 04:35:42,43091,13066,27281,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2...","ueRPE0CX75ePGMqOFVj6IQ, 52oH4DrRvzzl8wh5UXyU0A...",3138,...,264,184,157,251,1847,7054,3131,3131,1521,1946
2,2WnXYQFK0hXEoTxPtV2zvg,Steph,665,2008-07-25 10:41:00,2086,1010,1003,"2009,2010,2011,2012,2013","LuO3Bn4f3rlhyHIaNfTlnA, j9B4XdHUhDfTKVecyWQgyA...",52,...,13,10,17,3,66,96,119,119,35,18
3,SZDeASXq7o05mMNLshsdIA,Gwen,224,2005-11-29 04:38:33,512,330,299,"2009,2010,2011","enx1vVPnfdNUdPho6PH_wg, 4wOcvMLtU6a9Lslggq74Vg...",28,...,4,1,6,2,12,16,26,26,10,9
4,hA5lMy-EnncsH4JoR-hFGQ,Karen,79,2007-01-05 19:40:59,29,15,7,,"PBK4q9KEEBHhFvSXCUirIw, 3FWPpM7KU1gXeOM_ZbYMbA...",1,...,1,0,0,0,1,1,0,0,0,0


In [ ]:
#Datos de usuario de preprocesamiento
def preprocess_user_df(df):
    #Cambiar la identificación de usuario a algo más adecuado para nuestro análisis.
    global userid_to_idx
    userid_to_idx = {user : idx for idx, user in enumerate(df.user_id.unique())}
    df.user_id = df.user_id.map(lambda x: userid_to_idx[x])
    #Cambiar los ID de amigos a nuevos ID int
    df.friends = df.friends.map(lambda x : str([userid_to_idx[user] for user in x.split(',') if user in userid_to_idx]))
    #TDT hasta fechahora
    df.yelping_since = pd.to_datetime(df.yelping_since)
    df.elite = df.elite.replace('', np.nan)
    return df

user_df = preprocess_user_df(user_df)

In [ ]:
#Envío de datos de usuario a SQL db
user_df.to_sql('users', db, if_exists='append', index=False)
#Liberar memoria
del user_df

In [ ]:
#Aquí cargamos datos de reseñas.

#El tamaño de review.json domina la RAM, por lo tanto, procesaremos los datos en fragmentos y los almacenaremos en la tabla.
for data in load_rows_gen(REVIEW_PATH):
    #transformaciones
    data.user_id = data.user_id.apply(lambda key : userid_to_idx[key] if key in userid_to_idx else np.nan)
    data.business_id = data.business_id.apply(lambda key : businessid_to_idx[key] if key in businessid_to_idx else np.nan)
    data.date = pd.to_datetime(data.date)
    data.dropna(inplace=True)
    #enviando sql
    data.to_sql('reviews', db, if_exists='append', index=False)
#del data

Loaded chunk of size: 1000000 , Time = 48.38 secs.
Loaded chunk of size: 1000000 , Time = 20.01 secs.
Loaded chunk of size: 1000000 , Time = 23.2 secs.
Loaded chunk of size: 1000000 , Time = 65.1 secs.
Loaded chunk of size: 1000000 , Time = 94.72 secs.
Loaded chunk of size: 1000000 , Time = 24.96 secs.
Loaded chunk of size: 990280 , Time = 19.29 secs.
